# Notebook to explore the tables created by dune_source using the default config in this repo
Before running this notebook, run the pipeline to create the tables in the duckdb database.:
```
python duckdb_pipeline.py
```
After running the pipeline, the duckdb database will be populated with the following tables:
- dex_volume
- dex_volume_incremental
- y2k_price_data
- custom_sql (with incremental cursor)

Important: make sure to close the connection to the duckdb database after running the notebook, if you don't, the connection will be locked and you won't be able to run the pipeline again.

In [35]:
import duckdb
db = duckdb.connect("dune_source.duckdb")

In [37]:
# dex_volume
db.execute("select * from dune_queries.dex_volume order by _col1").fetchdf()

,project,_col1,usd_volume,_dlt_load_id,_dlt_id
0,gmx,2025-01-28 00:00:00.000 UTC,1.823032e+05,1740628817.4497652,PQsYSVVpXmyunA
1,0x-API,2025-01-28 00:00:00.000 UTC,3.051640e+07,1740628817.4497652,NkKeCzZI3gQgzA
2,swaap,2025-01-28 00:00:00.000 UTC,4.745830e+07,1740628817.4497652,7JsKHzbiKBwGog
3,soswap,2025-01-28 00:00:00.000 UTC,3.509940e+02,1740628817.4497652,ks5AvPj9kJcBJg
4,nile,2025-01-28 00:00:00.000 UTC,4.876336e+06,1740628817.4497652,G4WOQWEQ71vSsw
...,...,...,...,...,...
4161,zebra,2025-02-26 00:00:00.000 UTC,2.145958e+04,1740628817.4497652,dcPsI0G5ogTPtA
4162,horizondex,2025-02-26 00:00:00.000 UTC,1.568853e+02,1740628817.4497652,rR+FzUlEVHI8ew
4163,mute,2025-02-26 00:00:00.000 UTC,1.042015e+05,1740628817.4497652,I4+PPHJPTiLARA
4164,solidlizard,2025-02-26 00:00:00.000 UTC,1.561109e+01,1740628817.4497652,9oAuwbrbzXL+4A


In [38]:
# notice that the date column goes back to 2025-01-01 (starting_replication_value)
db.execute("select * from dune_queries.dex_volume_incremental order by date").fetchdf() 

,project,date,usd_volume,_dlt_load_id,_dlt_id
0,spiritswap,2025-01-01 00:00:00.000 UTC,1.560392e+05,1740628577.099942,NIC1qAYGRgqYxg
1,mummy_finance,2025-01-01 00:00:00.000 UTC,1.835366e+03,1740628577.099942,xcAIyooYxVfHzg
2,infusion,2025-01-01 00:00:00.000 UTC,1.061927e+04,1740628577.099942,uvWKszbHFktwGw
3,zyberswap,2025-01-01 00:00:00.000 UTC,3.962155e+02,1740628577.099942,WVhsbxaXb3BY3w
4,derpdex,2025-01-01 00:00:00.000 UTC,3.514931e+02,1740628577.099942,1bnQliscnOIdYA
...,...,...,...,...,...
7987,ramses,2025-02-27 00:00:00.000 UTC,5.257255e+05,1740628577.099942,hK+/wT+m2wIDYg
7988,shadow,2025-02-27 00:00:00.000 UTC,1.558703e+07,1740628577.099942,uOMmN5N5LybOGA
7989,quickswap,2025-02-27 00:00:00.000 UTC,4.413531e+06,1740628577.099942,VjkNzEpzZMyeMw
7990,soswap,2025-02-27 00:00:00.000 UTC,8.038838e-01,1740628577.099942,AUUhDT+qrdZyXA


In [39]:
# y2k_price_data (pass in Y2K as symbol to query param)
db.execute("select * from dune_queries.y2k_price_data").fetchdf()

,timestamp,year,month,blockchain,contract_address,symbol,decimals,price,_dlt_load_id,_dlt_id
0,2022-09-23 00:00:00.000 UTC,2022,9,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000031,1740628817.4497652,uFMiHwGLsgCJ0Q
1,2022-09-25 00:00:00.000 UTC,2022,9,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000030,1740628817.4497652,uPL1mVq28DSw2Q
2,2022-09-27 00:00:00.000 UTC,2022,9,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000030,1740628817.4497652,bGRnK4gqgvO32Q
3,2022-10-23 00:00:00.000 UTC,2022,10,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000030,1740628817.4497652,OC+rtP+gaWDEiQ
4,2022-10-24 00:00:00.000 UTC,2022,10,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,0.000030,1740628817.4497652,1wUexT2vyQ0xPA
...,...,...,...,...,...,...,...,...,...,...
4459,2024-12-21 00:00:00.000 UTC,2024,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.924233,1740628817.4497652,grU0JyDCnEVGkg
4460,2024-12-27 00:00:00.000 UTC,2024,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.575739,1740628817.4497652,SOnCO0W5/y784Q
4461,2025-01-06 00:00:00.000 UTC,2025,1,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.706050,1740628817.4497652,1MwZ9DkoPblL3A
4462,2025-01-27 00:00:00.000 UTC,2025,1,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.382923,1740628817.4497652,xFjzL4H2H0YFtg


In [40]:
# custom_sql
db.execute("select * from dune_queries.custom_sql order by timestamp").fetchdf()

,timestamp,blockchain,contract_address,symbol,price,_dlt_load_id,_dlt_id
0,2024-11-02 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.083926,1740628577.099942,JRaNXE1gePxzcw
1,2024-11-03 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.079143,1740628577.099942,bV0lo4c1c04vBQ
2,2024-11-04 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.078314,1740628577.099942,1QTZXvCPW03QHQ
3,2024-11-05 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.079181,1740628577.099942,8M5cHzSP7ARrdQ
4,2024-11-06 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.089753,1740628577.099942,YKAFwzNKIvQtIg
...,...,...,...,...,...,...,...
111,2025-02-22 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.050081,1740628577.099942,ImrOaqTl8FtQOg
112,2025-02-23 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.048287,1740628577.099942,U2i4ZPFfWrL1TQ
113,2025-02-24 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.044149,1740628577.099942,6uN1qwM92HH+bA
114,2025-02-25 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.038989,1740628577.099942,5GxFWA07zffnXA


In [41]:
# close the connection
db.close()